<h1 align="center">SBERT vs. Data2vec na classificação de texto</h1>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

Neste script estudaremos o uso de dois dos populares modelos pré-treinados do Hugging Face: `SBERT` vs `Data2vec`.

Este estudo foi baseado no artigo do [Data Scientist Jinhang Jiang](https://towardsdatascience.com/sbert-vs-data2vec-on-text-classification-e3c35b19c949).

# Introdução

<font color="orange">Pessoalmente, acredito que todas as pesquisas sofisticadas de ML e trabalhos avançados de algoritmo de IA têm um valor mínimo, se não zero, até a data em que podem ser aplicados a projetos da vida real sem solicitar aos usuários uma quantidade insana de recursos e conhecimento excessivo do domínio. E [Hugging Face](https://huggingface.co/) constrói a ponte. Hugging Face é o lar de milhares de modelos pré-treinados que fizeram grandes contribuições para democratizar a inteligência artificial por meio de código aberto e ciência aberta. 

Hoje, quero oferecer a você uma demonstração de código de ponta a ponta para comparar `dois dos modelos pré-treinados mais populares`, conduzindo uma análise de `Classificação de texto` com vários rótulos.
</font>

<font color="red">O primeiro modelo</font> é o [SentenceTransformers (SBERT)](https://www.sbert.net/). Na verdade, trata-se de uma coleção de vários modelos pré-treinados para várias tarefas criadas pela equipe do [Ubiquitous Knowledge Processing Lab na Technical University Darmstadt](https://www.informatik.tu-darmstadt.de/ukp/ukp_home/index.en.jsp). Eu usei o `SBERT` várias vezes em meus projetos anteriores. Eles ainda têm uma biblioteca python que oferece a flexibilidade de não usar a `API Hugging Face` e a `estrutura Pytorch`. Confira [aqui](https://arxiv.org/abs/2202.03555).

<font color="red">O segundo modelo</font> é o [Data2vec](https://ai.facebook.com/blog/the-first-high-performance-self-supervised-algorithm-that-works-for-speech-vision-and-text/), um poderoso modelo pré-treinado oferecido pela equipe de `IA` da `Meta` (Facebook). É uma estrutura `auto-supervisionada` (modelo de professor → modelo de aluno) projetada para codificação (Encoding) de texto, áudio e imagem. Se você estiver interessado em saber como ele é desenvolvido, poderá encontrar o artigo original [aqui](https://arxiv.org/abs/2202.03555).

# Bibliotecas necessárias

In [ ]:
!pip install torch transformers memory_profiler datasets

In [2]:
import time
import datetime
tic = time.time()



import torch
import random
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

from datasets import load_metric
%load_ext memory_profiler

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

# Data

Para os dados, estou usando um conhecido conjunto de dados de texto de código aberto: `BBC News Group` (sua licença está aqui: https://opendatacommons.org/licenses/dbcl/1-0/ ). Você pode carregar os dados fazendo o seguinte:

In [3]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(subset="all", 
                             shuffle=True, 
                             remove=("headers", "footers", "quotes"))
documents = dataset.data
labels = dataset.target

In [4]:
#documents

In [5]:
#labels

In [11]:
path = "/content/drive/MyDrive/6_Hugging_Face_NLP-Computer_Vision-SpeechProcessing/12_Zero-Shot-Text-Classification-with-Hugging-Face/2_SBERT_vs_Data2vec_on_Text_Classification"

df = pd.read_csv(path + "/bbc-text.csv")

df.head(6)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
5,politics,howard hits back at mongrel jibe michael howar...


In [12]:
df.shape

(2225, 2)

In [21]:
print(df.category.unique())
print("")
# Função "factorize" converte valores CATEGÓRICOS em valores NUMÉRICOS
print(pd.factorize(df.category)[1]) # Retorna uma Tupla
print("")
print(pd.factorize(df.category)[0]) # Aqui obteremos uma Representação única de valores numéricos

['tech' 'business' 'sport' 'entertainment' 'politics']

Index(['tech', 'business', 'sport', 'entertainment', 'politics'], dtype='object')

[0 1 2 ... 3 4 2]


# Definir Semente na Frente

In [20]:
def set_seed(seed: int):
  random.seed(seed)
  np.random.seed(seed)
  if is_torch_available():
      torch.manual_seed(seed)
      torch.cuda.manual_seed_all(seed)

  if is_tf_available():
      import tensorflow as tf
 
      tf.random.set_seed(seed)
 
set_seed(42)


# Construir classificador com Transformer

Algumas observações no código a seguir:

* <font color="red">Split de nossos Dados</font> 

<font color="orange">Preste atenção a um detalhe aqui: estou usando um arquivo `CSV` em vez de importar os dados do `sklearn`. Então, eu forneci os dados de entrada como uma lista (`X.tolist()`) . Sem fazer isso, o modelo lançará erros posteriormente.</font>


* <font color="red">Caregamos nosso Tokenizador</font>

<font color="orange">`model_name:` este parâmetro deve ser uma string com o nome do modelo pré-treinado que você deseja usar. Você pode encontrar os modelos disponíveis no [Models - Hugging Face](https://huggingface.co/models)</font>

\\
<font color="orange">`max_length:` este parâmetro afetará diretamente o tempo de treinamento e a velocidade de treinamento. Você deve de especificar o comprimento do texto que deseja que o modelo processe para cada documento se cada documento for muito longo.</font>

\\
<font color="orange">`padding:` defina este parâmetro como `True` se você tiver dado um `max_length`. Preencher para a sequência mais longa no lote (ou sem preenchimento se apenas uma única sequência for fornecida).</font>


* <font color="red">Instanciamos nosso Modelo</font>

<font color="orange">`AutoModelForSequenceClassification:` “AutoModel…” ajudará você a identificar automaticamente o modelo correto a ser usado. Até agora, ele funciona bem para mim. “…ForSequenceClassification” é usado `especificamente para problemas de classificação`.</font>

\\
<font color="orange">`to(“cuda”):` se a `GPU` estiver disponível em sua máquina, você pode anexar esta função no final para aproveitar o poder da `GPU`. Sem esta função, o tempo de treinamento normalmente aumentará significativamente.</font>


* <font color="red">Load Metrics</font>

<font color="orange">`metrics_name:` deve ser uma string. Para nossa demonstração, escolhi `“f1”` como a métrica de avaliação. Você pode encontrar as opções disponíveis aqui: https://github.com/huggingface/datasets/tree/master/metrics, mas recomendo ler o mais atualizado aqui: https://huggingface.co/docs/evaluate/a_quick_tour</font>

\\
<font color="orange">`average:` passei este parâmetro porque estou usando `f1 score` para avaliar um problema de `classificação multi-rótulo`. NÃO é um parâmetro universal.</font>

In [ ]:
def TextClassification_with_Transformer(model_name: str, Data: pd.Series, Target:pd.Series, test_size: np.float64, max_length: int, num_labels: int, num_epochs: int, metrics_name: str):

  # Nossos Dados (Features e Target)
  X = Data
  y = Target
  y = pd.factorize(y)[0] # Aqui obteremos uma Representação única de valores numéricos

  # Load Metrics
  metric = load_metric(metrics_name) # Por exemplo: 'f1'

  # Split de nossos Dados
  X_train, X_test, y_train, y_test = train_test_split(X.tolist(), y, test_size=test_size)

  # Carregamos nosso Tokenizador
  tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

  # Encode (Codificando) o Texto
  train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
  valid_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)


  class MakeTorchData(torch.utils.data.Dataset):
      def __init__(self, encodings, labels):
          self.encodings = encodings
          self.labels = labels

      def __getitem__(self, idx):
          item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
          item["labels"] = torch.tensor([self.labels[idx]])
          return item

      def __len__(self):
          return len(self.labels)


  # Converter nossos Dados Tokenizados em um Dataset torch
  train_dataset = MakeTorchData(train_encodings, y_train.ravel()) # .ravel() --> Função NumPy, transforma arrays Multidimensionais em arrays Unidimensionais
  valid_dataset = MakeTorchData(valid_encodings, y_test.ravel())


  # Instanciamos nosso Modelo
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_labels).to("cuda")

  # Cálculo de Métricas
  def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # 'micro', 'macro', etc. are for multi-label classification. If you are running a binary classification, leave it as default or specify "binary" for average
    return metric.compute(predictions=predictions, references=labels, average="micro")  

  # Especificamos os argumentos para o trainer  
  training_args = TrainingArguments(
      output_dir='./results',          # output directory
      num_train_epochs=num_epochs,     # total number of training epochs
      per_device_train_batch_size=8,   # batch size per device during training
      per_device_eval_batch_size=20,   # batch size for evaluation
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
      metric_for_best_model = metrics_name,    # select the base metrics
      logging_steps=200,               # log & save weights each logging_steps
      save_steps=200,
      evaluation_strategy="steps",     # evaluate each `logging_steps`
  ) 

  # Call the Trainer
  trainer = Trainer(
      model=model,                         # the instantiated Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=valid_dataset,          # evaluation dataset
      compute_metrics=compute_metrics,     # the callback that computes metrics of interest
  )

  # Train the model
  trainer.train()
  
  # Call the summary
  trainer.evaluate()



  return trainer, model          


# Sentence BERT

In [ ]:
%%time
%%memit
sbert_trainer, sbert_model = TextClassification_with_Transformer(model_name = 'sentence-transformers/all-mpnet-base-v2', 
                                                                 Data = df.text, 
                                                                 Target = df.category, 
                                                                 test_size = 0.33, 
                                                                 max_length = 512, 
                                                                 num_labels = 5, 
                                                                 num_epochs = 5, 
                                                                 metrics_name='f1')



<ipython-input-11-178ec6b1d30d>:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric(metrics_name)


Some weights of the model checkpoint at sentence-transformers/all-mpnet-base-v2 were not used when initializing MPNetForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a

Step,Training Loss,Validation Loss,F1
200,1.249800,0.501773,0.983673


***** Running Evaluation *****
  Num examples = 735
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-200
Configuration saved in ./results/checkpoint-200/config.json
Model weights saved in ./results/checkpoint-200/pytorch_model.bin


Step,Training Loss,Validation Loss,F1
200,1.249800,0.501773,0.983673
400,0.252200,0.148193,0.967347
600,0.102400,0.453734,0.910204
800,0.053000,0.103773,0.979592


***** Running Evaluation *****
  Num examples = 735
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-400
Configuration saved in ./results/checkpoint-400/config.json
Model weights saved in ./results/checkpoint-400/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 735
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-600
Configuration saved in ./results/checkpoint-600/config.json
Model weights saved in ./results/checkpoint-600/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 735
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-800
Configuration saved in ./results/checkpoint-800/config.json
Model weights saved in ./results/checkpoint-800/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-200 (score: 0.9836734693877551).
***** Running Evaluation *****
  Num examples = 735
  Batch size = 20


peak memory: 4044.75 MiB, increment: 3167.53 MiB
CPU times: user 13min 37s, sys: 1min 39s, total: 15min 16s
Wall time: 15min 24s


# Data2vec

In [ ]:
%%time
%%memit
d2v_trainer, d2v_model = TextClassification_with_Transformer(model_name = 'facebook/data2vec-text-base', 
                                                             Data = df.text, 
                                                             Target = df.category, 
                                                             test_size = 0.33, 
                                                             max_length = 512, 
                                                             num_labels = 5, 
                                                             num_epochs = 5, 
                                                             metrics_name='f1')



In [ ]:

toc=time.time()
print(datetime.timedelta(seconds = toc-tic))
